In [ ]:
# ────────────────────────────────────────────────────────────────
# 0.  Environment setup  (run once)
# ────────────────────────────────────────────────────────────────
!pip install -q --upgrade insightface onnxruntime
!pip install -q opencv-python typing-extensions

# ────────────────────────────────────────────────────────────────
# 1.  Imports  &  model initialisation
# ────────────────────────────────────────────────────────────────
import cv2, numpy as np, os, ipywidgets as widgets
from google.colab import files
from insightface.app import FaceAnalysis          # high-level detector+embedder
from numpy.linalg import norm
from google.colab import drive
import shutil
from PIL import Image

drive.mount('/content/drive')

# use GPU 0 if it exists, otherwise CPU (ctx_id = -1)
CTX = 0 if os.environ.get("COLAB_GPU") else -1
app = FaceAnalysis(name="buffalo_l")              # detector: SCRFD; embedder: ArcFace
app.prepare(ctx_id=CTX, det_size=(640, 640))      # tweak det_size for speed vs. recall
print("→  Model loaded!")



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.0 MB/s eta 0:00:00
Mounted at /content/drive
download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:05<00:00, 55483.06KB/s]
/usr/local/lib/python3.12/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:123: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
→  Model loa

In [ ]:
# ────────────────────────────────────────────────────────────────
# 2.  Build a gallery of known people
#     • upload one clear photo per person
#     • you’ll type a name for each image when prompted
# ────────────────────────────────────────────────────────────────
gallery = []          # list of (embedding_vector, "Person Name")

print("\n📸  Upload ID photos (Press CANCEL when finished)…")
while True:
    up = files.upload()         # choose images; empty dict => user hit cancel
    if len(up) == 0:
        break
    for fname in up.keys():
        img = cv2.imread(fname)
        faces = app.get(img)
        if not faces:
            print(f"  – No face found in {fname} (skipped)")
            continue
        emb = faces[0].normed_embedding         # already L2-normalised
        label = input(f"Name for {fname}? → ")
        gallery.append((emb, label.strip()))
        print(f"  ✓  Added {label.strip()}")

print(f"\nGallery ready with {len(gallery)} identities!")




📸  Upload ID photos (Press CANCEL when finished)…



Gallery ready with 0 identities!


In [ ]:
# ────────────────────────────────────────────────────────────────
# 3.  Detect + recognise faces in new photos (Dynamic gallery update)
# ────────────────────────────────────────────────────────────────
THRESH = 0.15            # similarity threshold for recognition
IOU_THRESH = 0.4         # skip overlapping faces
unknown_counter = 1

def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    if interArea == 0:
        return 0.0
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    return interArea / float(boxAArea + boxBArea - interArea)

def update_embedding(old_emb, new_emb, alpha=0.5):
    """
    Simple running average update of embedding:
    new_emb_stored = alpha * old + (1-alpha) * new
    """
    return (alpha * old_emb + (1 - alpha) * new_emb)

print("\n🖼️  Upload test images to recognise…")
test = files.upload()
folder_name = "outputs"
folder_path = f"/content/drive/MyDrive/{folder_name}"
os.makedirs(folder_path, exist_ok=True)
print(f"Folder created at: {folder_path}")

for fname in test.keys():
    img  = cv2.imread(fname)
    draw = img.copy()
    print("Scanning", fname)
    maxFaceCoords = [0,0,0,0]
    maxName = ""
    processed_faces = []

    faces = app.get(img)

    for face in faces:
        x1, y1, x2, y2 = face.bbox.astype(int)
        x1, x2 = sorted([x1, x2])
        y1, y2 = sorted([y1, y2])
        h, w = img.shape[:2]
        x1, x2 = max(0, x1), min(w, x2)
        y1, y2 = max(0, y1), min(h, y2)
        if x2 <= x1 or y2 <= y1:
            continue

        current_box = [x1, y1, x2, y2]
        skip = False
        for prev_box in processed_faces:
            if compute_iou(current_box, prev_box) > IOU_THRESH:
                skip = True
                break
        if skip:
            continue
        processed_faces.append(current_box)

        emb = face.normed_embedding
        sims = [float(np.dot(emb, g[0])) for g in gallery]
        name = "None"

        if sims and max(sims) > THRESH:
            # Existing face -> dynamically update embedding
            idx = int(np.argmax(sims))
            name = gallery[idx][1]
            if face.kps is not None and not np.any(np.isnan(face.kps)):
              updated_emb = update_embedding(gallery[idx][0], emb)
              gallery[idx] = (updated_emb, name)
              print(f"  🔄 Updated embedding for {name}")
        else:
            # New unknown face
            if face.kps is not None and not np.any(np.isnan(face.kps)):
              name = f"Unknown_{unknown_counter:03d}"
              unknown_counter += 1
              gallery.append((emb, name))
              cropped = img[y1:y2, x1:x2]
              cv2.imwrite(f"{folder_path}/{name}.jpg", cropped)
              print(f"  🆕 Added new face to gallery as {name}")

        if name != "None":
          print(f"  The face at ({x1},{y1},{x2},{y2}) is: {name}")
          # draw results
          color = (0, 0, 255)  # blue
          cv2.rectangle(draw, (x1, y1), (x2, y2), color, 2)
          cv2.putText(draw, name, (x1, y1 - 10),
              cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2
          )
          if (x2-x1)*(y2-y1) > (maxFaceCoords[2]-maxFaceCoords[0])*(maxFaceCoords[3]-maxFaceCoords[1]):
              maxFaceCoords = [x1, y1, x2, y2]
              maxName = name
        else:
          print(f"  The face at ({x1},{y1},{x2},{y2}) has been filtered out")

    # Draw the biggest face for reference
    color = (0, 255, 0)
    cv2.rectangle(draw, tuple(maxFaceCoords[:2]), tuple(maxFaceCoords[2:]), color, 2)
    cv2.putText(draw, maxName, (maxFaceCoords[0], maxFaceCoords[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out_name = f"annotated_{os.path.basename(fname)}"
    cv2.imwrite(out_name, draw)
    draw = cv2.cvtColor(draw, cv2.COLOR_RGB2BGR)
    draw = Image.fromarray(draw)
    draw.save(f"{folder_path}/{out_name}")

# Zip the output folder
zip_path = f"/content/{folder_name}.zip"
shutil.make_archive(folder_name, 'zip', folder_path)
files.download(zip_path)
print("✅ All done!")


🖼️  Upload test images to recognise…


Streaming output truncated to the last 5000 lines.
  The face at (392,53,490,185) is: Unknown_011
  🔄 Updated embedding for Unknown_023
  The face at (313,116,386,208) is: Unknown_023
  🔄 Updated embedding for Unknown_013
  The face at (746,25,854,170) is: Unknown_013
Scanning elevator_4190 (2).jpg
  🔄 Updated embedding for Unknown_014
  The face at (35,116,115,219) is: Unknown_014
  🔄 Updated embedding for Unknown_020
  The face at (555,158,623,237) is: Unknown_020
  🔄 Updated embedding for Unknown_011
  The face at (392,54,490,185) is: Unknown_011
  🔄 Updated embedding for Unknown_016
  The face at (119,168,213,281) is: Unknown_016
  🔄 Updated embedding for Unknown_023
  The face at (309,117,382,207) is: Unknown_023
  🔄 Updated embedding for Unknown_013
  The face at (747,26,854,171) is: Unknown_013
Scanning elevator_4191 (2).jpg
  🔄 Updated embedding for Unknown_014
  The face at (35,116,113,220) is: Unknown_014
  🔄 Updated embedding for Unknown_020
  The face at (554,158,623,237) i

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ All done!


Use Office Surveillance Video for testing.

Search how to prevent masking.